In [ ]:
# import
import pickle
import numpy as np
import pandas as pd

In [ ]:
# data diectory, modify before use
data_dir = "scripts/0_sepsis_ground_truth/full_data"

In [ ]:
# patient id
separate_id = pickle.load(open(
    data_dir + "Final_ID.pickle",
    "rb"
))
nonsep_id, sepsis_id = separate_id['nonsep'], separate_id['sepsis']
all_id = nonsep_id + sepsis_id

In [ ]:
# patinet_info
print("sepsis: ", len(sepsis_id))
print("nonsep: ", len(nonsep_id))
print("all: ", len(sepsis_id) + len(nonsep_id))

In [ ]:
# ============================================= 12 h =======================================================

In [ ]:
# sepsis
# filter id to find all where sepsis offset is after 12 h of admission.
horizon = 12
nan_vital = []
short_horizon = []
good_to_use = []
for p_id in sepsis_id:
    # load data
    patient_data = pd.read_csv("processed_data/0_filled_lab/{}.csv".format(p_id), index_col=False)
    # admission time
    adm_time = patient_data['offset'].min()
    # sepsis onset
    sep_time = patient_data['offset'].loc[patient_data['label'] == 1].min()
    # too short, < horizon
    if sep_time - adm_time < 60 * horizon:
        short_horizon.append(p_id)
        continue
    # reset offset time
    patient_data['offset'] = patient_data['offset'] - sep_time
    # get data within horizon
    patient_data = patient_data.loc[
        (patient_data['offset'] <= 0) & (patient_data['offset'] >= -horizon * 60 - 30)
    ]
    # too short, check again
    if patient_data['offset'].min() > -60 * horizon + 60:
        short_horizon.append(p_id)
        continue
    # check nan in vitals
    if any([
        patient_data['heartrate'].loc[patient_data['heartrate'].isna()].shape[0] > 0.3 * patient_data.shape[0],
        patient_data['respiration'].loc[patient_data['respiration'].isna()].shape[0] > 0.3 * patient_data.shape[0],
        patient_data['noninvasivesystolic'].loc[patient_data['noninvasivesystolic'].isna()].shape[0] > 0.3 * patient_data.shape[0],
        patient_data['noninvasivediastolic'].loc[patient_data['noninvasivediastolic'].isna()].shape[0] > 0.3 * patient_data.shape[0],
        patient_data['noninvasivemean'].loc[patient_data['noninvasivemean'].isna()].shape[0] > 0.3 * patient_data.shape[0],
    ]):
        nan_vital.append(p_id)
        continue
    # otherwise, it's good to use
    good_to_use.append(p_id)
    patient_data.to_csv("processed_data/1_{}_h_pass_nan_check/{}.csv".format(horizon, p_id), index=False)
print(len(nan_vital), len(short_horizon), len(good_to_use))
sepsis_test_id = good_to_use

In [ ]:
# nonsep
# filter id to find all where the end time is after 12 h of admission.
horizon = 12
nan_vital = []
short_horizon = []
good_to_use = []
for p_id in nonsep_id:
    # load data
    patient_data = pd.read_csv("processed_data/0_filled_lab/{}.csv".format(p_id), index_col=False)
    # admission time
    adm_time = patient_data['offset'].min()
    # end time
    end_time = patient_data['offset'].loc[patient_data['heartrate'].notna()].max()
    # too short, < horizon
    if end_time - adm_time < 60 * horizon:
        short_horizon.append(p_id)
        continue
    # reset end time
    patient_data['offset'] = patient_data['offset'] - end_time
    # get data within horizon
    patient_data = patient_data.loc[
        (patient_data['offset'] <= 0) & (patient_data['offset'] >= -horizon * 60 - 30)
    ]
    # too short, check again
    if patient_data['offset'].min() > -60 * horizon + 30:
        short_horizon.append(p_id)
        continue
    # check nan in vitals
    if any([
        patient_data['heartrate'].loc[patient_data['heartrate'].isna()].shape[0] > 0.3 * patient_data.shape[0],
        patient_data['respiration'].loc[patient_data['respiration'].isna()].shape[0] > 0.3 * patient_data.shape[0],
        patient_data['noninvasivesystolic'].loc[patient_data['noninvasivesystolic'].isna()].shape[0] > 0.3 * patient_data.shape[0],
        patient_data['noninvasivediastolic'].loc[patient_data['noninvasivediastolic'].isna()].shape[0] > 0.3 * patient_data.shape[0],
        patient_data['noninvasivemean'].loc[patient_data['noninvasivemean'].isna()].shape[0] > 0.3 * patient_data.shape[0],
    ]):
        nan_vital.append(p_id)
        continue
    # otherwise, it's good to use
    good_to_use.append(p_id)
print(len(nan_vital), len(short_horizon), len(good_to_use))
nonsep_test_id = good_to_use
# randomly select len(sepsis_test_id) patients
np.random.seed(1)
nonsep_ind = np.random.choice(range(len(nonsep_test_id)), len(sepsis_test_id), False)
nonsep_test_id = [
    nonsep_test_id[i] for i in nonsep_ind
]
print(len(nonsep_test_id))
# do this again and save data
for p_id in nonsep_test_id:
    # load data
    patient_data = pd.read_csv("processed_data/0_filled_lab/{}.csv".format(p_id), index_col=False)
    # admission time
    adm_time = patient_data['offset'].min()
    # end time
    end_time = patient_data['offset'].loc[patient_data['heartrate'].notna()].max()
    # reset end time
    patient_data['offset'] = patient_data['offset'] - end_time
    # get data within horizon
    patient_data = patient_data.loc[
        (patient_data['offset'] <= 0) & (patient_data['offset'] >= -horizon * 60 - 30)
    ]
    patient_data.to_csv("processed_data/1_{}_h_pass_nan_check/{}.csv".format(horizon, p_id), index=False)


In [ ]:
pickle.dump({
    'sepsis': sepsis_test_id, 'nonsep': nonsep_test_id
}, open("processed_data/ids/12h_ID.pickle", "wb"))

In [ ]:
# ============================================= 1 h =======================================================

In [ ]:
separate_id = pickle.load(open("processed_data/ids/12h_ID.pickle", "rb"))
sepsis_test_id = separate_id['sepsis']
nonsep_test_id = separate_id['nonsep']

In [ ]:
horizon = 1

In [ ]:
# sepsis
# filter id to find all where sepsis offset is after 1 h of admission, apart from 12 h patients
nan_vital = []
short_horizon = []
good_to_use = []
for p_id in sepsis_id:
    if p_id in sepsis_test_id:
        continue
    # load data
    patient_data = pd.read_csv("processed_data/0_filled_lab/{}.csv".format(p_id), index_col=False)
    # admission time
    adm_time = patient_data['offset'].min()
    # sepsis onset
    sep_time = patient_data['offset'].loc[patient_data['label'] == 1].min()
    # too short, < horizon
    if sep_time - adm_time < 60 * horizon:
        short_horizon.append(p_id)
        continue
    # reset offset time
    patient_data['offset'] = patient_data['offset'] - sep_time
    # get data within horizon
    patient_data = patient_data.loc[
        (patient_data['offset'] <= 0) & (patient_data['offset'] >= -horizon * 60 - 30)
    ]
    # too short, check again
    if patient_data['offset'].min() > -60 * horizon + 30:
        short_horizon.append(p_id)
        continue
    # check nan in vitals
    if any([
        patient_data['heartrate'].loc[patient_data['heartrate'].isna()].shape[0] > 0.3 * patient_data.shape[0],
        patient_data['respiration'].loc[patient_data['respiration'].isna()].shape[0] > 0.3 * patient_data.shape[0],
        patient_data['noninvasivesystolic'].loc[patient_data['noninvasivesystolic'].isna()].shape[0] > 0.3 * patient_data.shape[0],
        patient_data['noninvasivediastolic'].loc[patient_data['noninvasivediastolic'].isna()].shape[0] > 0.3 * patient_data.shape[0],
        patient_data['noninvasivemean'].loc[patient_data['noninvasivemean'].isna()].shape[0] > 0.3 * patient_data.shape[0],
    ]):
        nan_vital.append(p_id)
        continue
    # otherwise, it's good to use
    good_to_use.append(p_id)
    patient_data.to_csv("processed_data/1_{}_h_pass_nan_check/{}.csv".format(horizon, p_id), index=False)
print(len(nan_vital), len(short_horizon), len(good_to_use))
sepsis_train_id = good_to_use

In [ ]:
# nonsep
# filter id to find all where the end time is after 12 h of admission.
nan_vital = []
short_horizon = []
good_to_use = []
for p_id in nonsep_id:
    if p_id in nonsep_test_id:
        continue
    # load data
    patient_data = pd.read_csv("processed_data/0_filled_lab/{}.csv".format(p_id), index_col=False)
    # admission time
    adm_time = patient_data['offset'].min()
    # end time
    end_time = patient_data['offset'].loc[patient_data['heartrate'].notna()].max()
    # too short, < horizon
    if end_time - adm_time < 60 * horizon:
        short_horizon.append(p_id)
        continue
    # reset end time
    patient_data['offset'] = patient_data['offset'] - end_time
    # get data within horizon
    patient_data = patient_data.loc[
        (patient_data['offset'] <= 0) & (patient_data['offset'] >= -horizon * 60 - 30)
    ]
    # too short, check again
    if patient_data['offset'].min() > -60 * horizon + 30:
        short_horizon.append(p_id)
        continue
    # check nan in vitals
    if any([
        patient_data['heartrate'].loc[patient_data['heartrate'].isna()].shape[0] > 0.3 * patient_data.shape[0],
        patient_data['respiration'].loc[patient_data['respiration'].isna()].shape[0] > 0.3 * patient_data.shape[0],
        patient_data['noninvasivesystolic'].loc[patient_data['noninvasivesystolic'].isna()].shape[0] > 0.3 * patient_data.shape[0],
        patient_data['noninvasivediastolic'].loc[patient_data['noninvasivediastolic'].isna()].shape[0] > 0.3 * patient_data.shape[0],
        patient_data['noninvasivemean'].loc[patient_data['noninvasivemean'].isna()].shape[0] > 0.3 * patient_data.shape[0],
    ]):
        nan_vital.append(p_id)
        continue
    # otherwise, it's good to use
    good_to_use.append(p_id)
print(len(nan_vital), len(short_horizon), len(good_to_use))
nonsep_train_id = good_to_use
# randomly select len(sepsis_train_id) patients
# np.random.seed(1)
# nonsep_ind = np.random.choice(range(len(nonsep_train_id)), len(sepsis_train_id), False)
# nonsep_train_id = [
#     nonsep_train_id[i] for i in nonsep_ind
# ]
print(len(nonsep_train_id))
# do this again and save data
for p_id in nonsep_train_id:
    # load data
    patient_data = pd.read_csv("processed_data/0_filled_lab/{}.csv".format(p_id), index_col=False)
    # admission time
    adm_time = patient_data['offset'].min()
    # end time
    end_time = patient_data['offset'].loc[patient_data['heartrate'].notna()].max()
    # reset end time
    patient_data['offset'] = patient_data['offset'] - end_time
    # get data within horizon
    patient_data = patient_data.loc[
        (patient_data['offset'] <= 0) & (patient_data['offset'] >= -horizon * 60 - 30)
    ]
    patient_data.to_csv("processed_data/1_{}_h_pass_nan_check/{}.csv".format(horizon, p_id), index=False)

In [ ]:
pickle.dump({
    'sepsis': sepsis_train_id, 'nonsep': nonsep_train_id
}, open("processed_data/ids/{}h_ID.pickle".format(horizon), "wb"))

In [ ]:
# ============================================= 6 h =======================================================

In [ ]:
separate_id = pickle.load(open("processed_data/ids/12h_ID.pickle", "rb"))
sepsis_test_id = separate_id['sepsis']
nonsep_test_id = separate_id['nonsep']

In [ ]:
horizon = 6

In [ ]:
# sepsis
# filter id to find all where sepsis offset is after 1 h of admission, apart from 12 h patients
nan_vital = []
short_horizon = []
good_to_use = []
for p_id in sepsis_id:
    if p_id in sepsis_test_id:
        continue
    # load data
    patient_data = pd.read_csv("processed_data/0_filled_lab/{}.csv".format(p_id), index_col=False)
    # admission time
    adm_time = patient_data['offset'].min()
    # sepsis onset
    sep_time = patient_data['offset'].loc[patient_data['label'] == 1].min()
    # too short, < horizon
    if sep_time - adm_time < 60 * horizon:
        short_horizon.append(p_id)
        continue
    # reset offset time
    patient_data['offset'] = patient_data['offset'] - sep_time
    # get data within horizon
    patient_data = patient_data.loc[
        (patient_data['offset'] <= 0) & (patient_data['offset'] >= -horizon * 60 - 30)
    ]
    # too short, check again
    if patient_data['offset'].min() > -60 * horizon + 30:
        short_horizon.append(p_id)
        continue
    # check nan in vitals
    if any([
        patient_data['heartrate'].loc[patient_data['heartrate'].isna()].shape[0] > 0.3 * patient_data.shape[0],
        patient_data['respiration'].loc[patient_data['respiration'].isna()].shape[0] > 0.3 * patient_data.shape[0],
        patient_data['noninvasivesystolic'].loc[patient_data['noninvasivesystolic'].isna()].shape[0] > 0.3 * patient_data.shape[0],
        patient_data['noninvasivediastolic'].loc[patient_data['noninvasivediastolic'].isna()].shape[0] > 0.3 * patient_data.shape[0],
        patient_data['noninvasivemean'].loc[patient_data['noninvasivemean'].isna()].shape[0] > 0.3 * patient_data.shape[0],
    ]):
        nan_vital.append(p_id)
        continue
    # otherwise, it's good to use
    good_to_use.append(p_id)
    patient_data.to_csv("processed_data/1_{}_h_pass_nan_check/{}.csv".format(horizon, p_id), index=False)
print(len(nan_vital), len(short_horizon), len(good_to_use))
sepsis_train_id = good_to_use

In [ ]:
# nonsep
# filter id to find all where the end time is after 12 h of admission.
nan_vital = []
short_horizon = []
good_to_use = []
for p_id in nonsep_id:
    if p_id in nonsep_test_id:
        continue
    # load data
    patient_data = pd.read_csv("processed_data/0_filled_lab/{}.csv".format(p_id), index_col=False)
    # admission time
    adm_time = patient_data['offset'].min()
    # end time
    end_time = patient_data['offset'].loc[patient_data['heartrate'].notna()].max()
    # too short, < horizon
    if end_time - adm_time < 60 * horizon:
        short_horizon.append(p_id)
        continue
    # reset end time
    patient_data['offset'] = patient_data['offset'] - end_time
    # get data within horizon
    patient_data = patient_data.loc[
        (patient_data['offset'] <= 0) & (patient_data['offset'] >= -horizon * 60 - 30)
    ]
    # too short, check again
    if patient_data['offset'].min() > -60 * horizon + 30:
        short_horizon.append(p_id)
        continue
    # check nan in vitals
    if any([
        patient_data['heartrate'].loc[patient_data['heartrate'].isna()].shape[0] > 0.3 * patient_data.shape[0],
        patient_data['respiration'].loc[patient_data['respiration'].isna()].shape[0] > 0.3 * patient_data.shape[0],
        patient_data['noninvasivesystolic'].loc[patient_data['noninvasivesystolic'].isna()].shape[0] > 0.3 * patient_data.shape[0],
        patient_data['noninvasivediastolic'].loc[patient_data['noninvasivediastolic'].isna()].shape[0] > 0.3 * patient_data.shape[0],
        patient_data['noninvasivemean'].loc[patient_data['noninvasivemean'].isna()].shape[0] > 0.3 * patient_data.shape[0],
    ]):
        nan_vital.append(p_id)
        continue
    # otherwise, it's good to use
    good_to_use.append(p_id)
print(len(nan_vital), len(short_horizon), len(good_to_use))
nonsep_train_id = good_to_use
# randomly select len(sepsis_train_id) patients
# np.random.seed(1)
# nonsep_ind = np.random.choice(range(len(nonsep_train_id)), len(sepsis_train_id), False)
# nonsep_train_id = [
#     nonsep_train_id[i] for i in nonsep_ind
# ]
print(len(nonsep_train_id))
# do this again and save data
for p_id in nonsep_train_id:
    # load data
    patient_data = pd.read_csv("processed_data/0_filled_lab/{}.csv".format(p_id), index_col=False)
    # admission time
    adm_time = patient_data['offset'].min()
    # end time
    end_time = patient_data['offset'].loc[patient_data['heartrate'].notna()].max()
    # reset end time
    patient_data['offset'] = patient_data['offset'] - end_time
    # get data within horizon
    patient_data = patient_data.loc[
        (patient_data['offset'] <= 0) & (patient_data['offset'] >= -horizon * 60 - 30)
    ]
    patient_data.to_csv("processed_data/1_{}_h_pass_nan_check/{}.csv".format(horizon, p_id), index=False)

In [ ]:
pickle.dump({
    'sepsis': sepsis_train_id, 'nonsep': nonsep_train_id
}, open("processed_data/ids/{}h_ID.pickle".format(horizon), "wb"))